#Group no: 11
### Team Members:
1) Aayush Parashar->25PGAI0002
2) Abhishek Sahoo->25PGAI0050
3) Divyakumar Sanghvi->25PGAI0035
4) Maulik Ruparel-> 25PGAI0024
5) Varsha Rai-> 25PGAI0049
6) Vishnu E.->25PGAI0145

Idea of the Project: To extract about IPL 2020 from the dataset. Dataset comprises the Twitter discourse surrounding the Indian Premier League (IPL) from August 15 to November 11, 2020. 

The data consists of 7 columns which are:


1) user_name (The username of the Twitter account that posted the tweet)
2) text (The actual tweet along with the hashtag)
3) user_followers (The number of followers the user had at the time of posting the
tweet)
4) date (The date when the tweet was posted)
5) user_location (location specified in the user's Twitter profile)
6) source (The device/platform/application used to post the tweet)
7) user_description (The bio or description provided by the user in their Twitter profile)

In [ ]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
sc = spark.sparkContext
#Reading the dataset
twit = sc.textFile('/FileStore/tables/jio2025/twitter_test2d.csv')



In [ ]:
#reviewing few entries
twit.take(3)

Out[3]: ['twit_no;user_name;text;user_followers;date;user_location;source;user_description',
 '0;Navaldeep Singh;Suresh raina has the best room out of all the players  #IPL2020;1311;25/08/20;Melbourne  New Delhi;Twitter for iPhone;#AppleFan #CricketAddict and a happy husband ',
 '1;Naseh Mohtesham;All of the Country is So Much into #IPL2020  This Year   Thanks BCCI  Best Decision To Host IPL This Year;165;29/09/20;;Twitter for iPhone;MBA  Cricket  Politics  RCB Freak ']

In [ ]:
# Removing header from the dataset
header = twit.first()

#Filtering out header row
twit_w_header = twit.filter(lambda x: x != header)
twit_w_header.take(3)


Out[4]: ['0;Navaldeep Singh;Suresh raina has the best room out of all the players  #IPL2020;1311;25/08/20;Melbourne  New Delhi;Twitter for iPhone;#AppleFan #CricketAddict and a happy husband ',
 '1;Naseh Mohtesham;All of the Country is So Much into #IPL2020  This Year   Thanks BCCI  Best Decision To Host IPL This Year;165;29/09/20;;Twitter for iPhone;MBA  Cricket  Politics  RCB Freak ',
 '2;Sports segment;#IPL2020 CSK win toss and bowl #IPL #RRvCSK;3;22/09/20;;Twitter Web App;The official Account of httpstcopJmosdM65g']

In [ ]:
#defining funtion to help us tokenise the literals of textfile
def tokenize_1(t):
    twit_no, user_name, text, user_followers, date, user_location, source, user_description = t.split(';')
    return (twit_no, user_name, text, user_followers, date, user_location, source, user_description)


In [ ]:
#funtion to extract month from the data in dataset
from datetime import datetime

def parse_date(line):
    date_str = line[4]
    try:
        date = datetime.strptime(date_str, "%d/%m/%y")
        return date.month
    except ValueError:
        print(f"Warning: could not parse date value {date_str}")
        return None



In [ ]:
#defining function to extracrt hastags from tweet texts in the dataset
import re


def extract_hashtags(twit_rows):
    tweet = twit_rows[2]
    tags = re.findall(r"#(\w+)", tweet)
    return [(parse_date(twit_rows), ('#' + tag.lower(), 1)) for tag in tags]


## applying functions

In [ ]:
#applying the function to textfile render values
twit_tokenize_1 = twit_w_header.map(tokenize_1)

In [ ]:
#reviewing tokens
twit_tokenize_1.take(4)

Out[9]: [('0',
  'Navaldeep Singh',
  'Suresh raina has the best room out of all the players  #IPL2020',
  '1311',
  '25/08/20',
  'Melbourne  New Delhi',
  'Twitter for iPhone',
  '#AppleFan #CricketAddict and a happy husband '),
 ('1',
  'Naseh Mohtesham',
  'All of the Country is So Much into #IPL2020  This Year   Thanks BCCI  Best Decision To Host IPL This Year',
  '165',
  '29/09/20',
  '',
  'Twitter for iPhone',
  'MBA  Cricket  Politics  RCB Freak '),
 ('2',
  'Sports segment',
  '#IPL2020 CSK win toss and bowl #IPL #RRvCSK',
  '3',
  '22/09/20',
  '',
  'Twitter Web App',
  'The official Account of httpstcopJmosdM65g'),
 ('3',
  'kamli kudi ',
  'Pooran Finally  #SaddaPunjab #IPL2020 #KXIP #RCBvKXIP',
  '29',
  '15/10/20',
  '',
  'Twitter for iPhone',
  'nan')]

In [ ]:
#making a rdd with (month,(hashtags,1))
twit_tokens_extracted=twit_tokenize_1.flatMap(lambda twit_rows:extract_hashtags(twit_rows))

In [ ]:
twit_tokens_extracted.take(2)

Out[11]: [(8, ('#ipl2020', 1)), (9, ('#ipl2020', 1))]

## Problem 1
###Find top 5 hashtag trends for different months. The results should be in the following 
###format:
###a. Month, rank, Hashtag, number of tweets

In [ ]:
#Building function to get top 5 ranks wise hastgas and number of tweets
def top_rank_tags(month):
    # Filter dataset to get specific month rows
    month_wise = twit_tokens_extracted.filter(lambda x: x[0] == month)
    
    # Get tuples in index 1 of extracted tokens
    month_wise_tags = month_wise.map(lambda rec: rec[1])
    
    # Get total number of tweets for a given hashtag in a given month
    month_wise_tags_list = (
        month_wise_tags.groupByKey()
        .mapValues(list)
        .mapValues(sum)
    )
    
    # Sort in descending order to get hashtags and their tweet counts
    month_wise_tags_list = month_wise_tags_list.sortBy(
        lambda x: x[1], ascending=False
    )
    
    # Get top 5 hashtags
    top_rank = month_wise_tags_list.take(5)
    
    # Format the result as a list of [month, hashtag, count] tuples
    result = list(map(lambda x: [month, x[0], x[1]], top_rank))
    
    return result


In [ ]:
#!pip install prettytable

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-aad643fc-02d1-4f64-a97d-3883ecd8010c/bin/python -m pip install --upgrade pip' command.


In [ ]:
#importing prettytable libary to present result in table form
from prettytable import prettytable

In [ ]:
# Final solution to problem 1
ans = []
# Loop will run on available months
for i in sorted(twit_tokens_extracted.groupByKey().mapValues(list).keys().collect()):
    result_prob_1 = top_rank_tags(i)
    result_prob_2 = [x.insert(1, result_prob_1.index(x) + 1) or x for x in result_prob_1]
    ans += result_prob_2

# Create a PrettyTable instance
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Month", "Rank", "Hashtag", "Number of Tweets"]

# Add rows to the table
for month, rank, hashtag, count in ans:
    table.add_row([month, rank, hashtag, count])

# Print the table
print(table)


+-------+------+-------------+------------------+
| Month | Rank |   Hashtag   | Number of Tweets |
+-------+------+-------------+------------------+
|   8   |  1   |   #ipl2020  |       1115       |
|   8   |  2   |     #ipl    |       197        |
|   8   |  3   |   #dream11  |       123        |
|   8   |  4   |     #csk    |       121        |
|   8   |  5   |  #iplinuae  |       105        |
|   9   |  1   |   #ipl2020  |      15537       |
|   9   |  2   |     #ipl    |       1470       |
|   9   |  3   |     #csk    |       1380       |
|   9   |  4   | #dream11ipl |       1255       |
|   9   |  5   |     #rcb    |       943        |
|   10  |  1   |   #ipl2020  |      12462       |
|   10  |  2   |     #csk    |       1447       |
|   10  |  3   | #dream11ipl |       1198       |
|   10  |  4   |     #ipl    |       1003       |
|   10  |  5   |    #kxip    |       869        |
|   11  |  1   |   #ipl2020  |       4733       |
|   11  |  2   |     #mi     |       656        |


## Problem 2 starts
### Find the trends of tweets (number of tweets) for each team over different months. The 
### results should have
###  a. Team name, month, number of tweets
###  b. A list of abbreviated names of the teams can be used to filter tweets (e.g. CSK, 
###        MI, RCB etc.)


In [ ]:
ipl_teams_2020 = [
    "MI",  # Mumbai Indians
    "CSK",  # Chennai Super Kings
    "RCB",  # Royal Challengers Bangalore
    "KKR",  # Kolkata Knight Riders
    "SRH",  # Sunrisers Hyderabad
    "DC",  # Delhi Capitals
    "KXIP",  # Kings XI Punjab
    "RR",  # Rajasthan Royals
]


In [ ]:
#Building function to get tuple month wise and pair-wise (team,1)
def extract_team_refs(twit_rows):
    team_includes = []
    tweet = twit_rows[2]
    for team in ipl_teams_2020:
        if team in tweet:
            team_includes.append([parse_date(twit_rows), (team, 1)])
    return team_includes

In [ ]:
#apply tokinazatinon function
team_wise_tweets = twit_tokenize_1.flatMap(lambda row: extract_team_refs(row))


In [ ]:
#Building function to get tokanized list of [team, month, count] tuples
def month_wise_teams_ref(month):
    # Filter dataset to get specific month rows
    month_wise_team_tweets = team_wise_tweets.filter(lambda x: x[0] == month)
    
    # Get tuples in index 1 of extracted tokens
    month_wise_teamtweets_tuples = month_wise_team_tweets.map(lambda rec: rec[1])
    
    # Get total number of tweets for a given team in a given month
    month_wise_teamtweets_rdd = (
        month_wise_teamtweets_tuples.groupByKey()
        .mapValues(list)
        .mapValues(sum)
    )
    
    # Sort in descending order to get team and their tweet counts
    month_wise_teamtweets_rdd = month_wise_teamtweets_rdd.sortBy(
        lambda x: x[1], ascending=False
    )
    
    # Get top 8 teams (hardcoded for total 8 teams)
    result_teams_list = month_wise_teamtweets_rdd.take(8)
    
    # Format the result as a list of [team, month, count] tuples
    result_teams = list(map(lambda x: [x[0], month, x[1]], result_teams_list))
    
    return result_teams


In [ ]:
# Final solution to problem 2
ans2 = []
# Loop will run on available teams
for i in sorted(twit_tokens_extracted.groupByKey().mapValues(list).keys().collect()):
    result_prob_2 = month_wise_teams_ref(i)
    result_prob_2 = [x for x in result_prob_2]
    ans2 += result_prob_2

# Create a PrettyTable instance
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Team", "Month", "Number of Tweets"]

# Add rows to the table
for team, month, count in ans2:
    table.add_row([team, month, count])

# Print the table
print(table)


+------+-------+------------------+
| Team | Month | Number of Tweets |
+------+-------+------------------+
| CSK  |   8   |       200        |
| RCB  |   8   |       182        |
| KKR  |   8   |        55        |
|  MI  |   8   |        54        |
|  DC  |   8   |        35        |
| KXIP |   8   |        29        |
| SRH  |   8   |        23        |
|  RR  |   8   |        22        |
| RCB  |   9   |       3985       |
| CSK  |   9   |       3576       |
|  MI  |   9   |       3275       |
|  RR  |   9   |       2724       |
| KXIP |   9   |       2611       |
| KKR  |   9   |       2030       |
| SRH  |   9   |       1904       |
|  DC  |   9   |       1768       |
| CSK  |   10  |       3690       |
| RCB  |   10  |       2811       |
|  DC  |   10  |       2646       |
|  RR  |   10  |       2604       |
| KXIP |   10  |       2562       |
| SRH  |   10  |       2375       |
|  MI  |   10  |       2181       |
| KKR  |   10  |       2175       |
|  DC  |   11  |       2022 

## Problem 3 starts
### Identify the top 5 tweets for each hashtag, ordered by the number of followers the 
### users have, who have tweeted. It should contain unique set of users. The result 
### should contain:
### a. Hashtag, list of 5 tweets, the username who has tweeted and the number of 
### followers

In [ ]:
import re #library for regex applications

#building function to get tokanized tuple in form of (#hastag,(twit_text,twit_no,user_followers))
def extract_hashtags_users(twit_rows):
    tweet = twit_rows[2]
    tags = re.findall(r"#(\w+)", tweet)
    return [
        ('#' + tag.lower(), (twit_rows[2], twit_rows[1], twit_rows[3]))
        for tag in tags
    ]


In [ ]:
#Building function to return only first 5 literals in a sorted list
def list_5_literals(line):
    list_sorted = sorted(line, key=lambda x: int(x[2]), reverse=True)
    unique_user_names = set(item[1] for item in list_sorted)
    unique_user_twits = []
    seen_users = set()
    
    for item in list_sorted:
        if item[1] not in seen_users:
            unique_user_twits.append(item)
            seen_users.add(item[1])
            if len(unique_user_twits) == 5:
                break
    
    return unique_user_twits


In [ ]:
twits_user_tags = twit_tokenize_1.flatMap(extract_hashtags_users) #getting key-value pair for (#tag,(Twit_text,User_name,User_followers))

In [ ]:
twits_user_tags_combine = twits_user_tags.groupByKey().mapValues(list) #getting key-value pair in form of (#tag,list(all->twit_text,User_name,User_followers))

In [ ]:
#solution to problem 3
twits_tag_wise_topfive_users = twits_user_tags_combine.map(lambda x: (x[0],list_5_literals(x[1])))
twits_tag_wise_topfive_users_list=twits_tag_wise_topfive_users.collect()
print(twits_tag_wise_topfive_users_list)

[('#rrvcsk', [('Match 4 63 R Jadeja to S Samson 6 runs 671 httpstcoFyhhx5FSSC #RRvCSK #Dream11IPL #IPL2020', 'IndianPremierLeague', '5403888'), ('#IPL2020kaDNA Debutant Yashasvi Jaiswal departs for 6 #RR vs #CSK #RRvCSK rajasthanroyals ChennaiIPL IPL httpstcoftpnNTFhEk', 'DNA', '2193966'), ('SIX Jofra Archer smashes a huge six off Lungi Ngidi sending it straight over his head#RRvCSK #RRvsCSK #IPL2020 httpstco7QcsRLb1d2', 'CricketNDTV', '1786792'), ('Air Samson #HallaBol  #RRvCSK  #IPL2020  IamSanjuSamson httpstcopXpKxuMgdc', 'Rajasthan Royals', '1232624'), ('#IPL2020 #RRvCSK #RRvsCSK Backtoback boundaries by Murali Vijay off Jaydev Unadkat CSK 160 in 22 oversFol httpstcoLEUTfAR3Cg', 'TOI Sports', '1027491')]), ('#saddapunjab', [('We believe #SaddaPunjab #IPL2020 #KXIP httpstcoEol8M5sbjo', 'Kings XI Punjab', '2204299'), ('Together we hustle together we win Great result and performance tonight We keep moving #SaddaPunjab httpstcoCVVlVpnzmc', 'Mayank Agarwal', '718902'), ('Morning run for

In [ ]:
# Print the results
for hashtag, twit_info in twits_tag_wise_topfive_users_list:
    print(f"Hashtag: {hashtag}")
    for twit_text, user_name, user_followers in twit_info:
        print(
            f"Tweet Content: {twit_text}\n"
            f"By: {user_name}, Followers: {user_followers}"
        )
    print("\n")


Hashtag: #rrvcsk
Tweet Content: Match 4 63 R Jadeja to S Samson 6 runs 671 httpstcoFyhhx5FSSC #RRvCSK #Dream11IPL #IPL2020
By: IndianPremierLeague, Followers: 5403888
Tweet Content: #IPL2020kaDNA Debutant Yashasvi Jaiswal departs for 6 #RR vs #CSK #RRvCSK rajasthanroyals ChennaiIPL IPL httpstcoftpnNTFhEk
By: DNA, Followers: 2193966
Tweet Content: SIX Jofra Archer smashes a huge six off Lungi Ngidi sending it straight over his head#RRvCSK #RRvsCSK #IPL2020 httpstco7QcsRLb1d2
By: CricketNDTV, Followers: 1786792
Tweet Content: Air Samson #HallaBol  #RRvCSK  #IPL2020  IamSanjuSamson httpstcopXpKxuMgdc
By: Rajasthan Royals, Followers: 1232624
Tweet Content: #IPL2020 #RRvCSK #RRvsCSK Backtoback boundaries by Murali Vijay off Jaydev Unadkat CSK 160 in 22 oversFol httpstcoLEUTfAR3Cg
By: TOI Sports, Followers: 1027491


Hashtag: #saddapunjab
Tweet Content: We believe #SaddaPunjab #IPL2020 #KXIP httpstcoEol8M5sbjo
By: Kings XI Punjab, Followers: 2204299
Tweet Content: Together we hustle together

## Prob 4 starts
###Create an inverted index of hashtags and the tweets so that given a hashtag, you can 
###print the list of tweets with the hashtags sorted by the months. The result should 
###contain
###   a. Hashtag, month, list of tweets (actual text)


In [ ]:
#defining function to extract hastags from tweet texts in the dataset
import re


def extract_hashtags_text(twit_rows):
    tweet = twit_rows[2]
    tags = re.findall(r"#(\w+)", tweet)
    return [
        ('#' + tag.lower(), (parse_date(twit_rows), twit_rows[2]))
        for tag in tags
    ]


In [ ]:
tag_month_twit=twit_tokenize_1.flatMap(extract_hashtags_text) #tokenizing inital data to grab required data in form of key-value pair (#tag,(month,twit_text))

In [ ]:
tag_month_twit_tagwise=tag_month_twit.groupByKey().mapValues(list) #Grouping data based on key to get the data in form of key-value pair-> (#tag,list(twit_text))

In [ ]:
def list_sort_monthwise(line):
    to_sort_list = line[1]
    list_sorted = sorted(to_sort_list,key= lambda x: x[0])
    return (line[0],list_sorted)

In [ ]:
#applying tokenization function 
tag_month_twit_monthwise=tag_month_twit_tagwise.map(list_sort_monthwise) #Sorting the key-value pair based on 

In [ ]:
tag_month_twit_monthwise.take(2)

Out[34]: [('#rrvcsk',
  [(9, '#IPL2020 CSK win toss and bowl #IPL #RRvCSK'),
   (9,
    'Predict the expected run by rajasthanroyals #IPL2020 #RRvCSK #PredictionContest #HallaBol #Yellove'),
   (9, 'Wholl get his century first #RRvCSK #IPL2020 #Yellove #CSKvRR'),
   (9,
    'I want Samson playing for India in all formats ASAP#IPL2020 #RRvCSK rajasthanroyals'),
   (9, 'Today match RR Vs CSK cricbuzz #RRvCSK #ipl2020'),
   (9, 'MSD not Archer #RRvCSK #IPL2020'),
   (9, 'Samson you beauty  #IPL2020 #RRvCSK #Dream11'),
   (9, 'Shitty umpiring #IPL2020 #IPLinUAE #RRvCSK ##rr #Csk'),
   (9,
    'Jofstradamus has a tweet for everything  #IPL2020 #RRvCSK httpstcoDaVoi3erzC'),
   (9,
    'Pic says everything  #CSKvsRR #RRvCSK #CSK #Cricket #IPL2020 #HallaBol #RCB #RCBvSRH #PlayBold #UmeshYadav httpstcoLMq2MqLlIy'),
   (9,
    'Poll time #IPL2020 #RRvCSK #CSKvsRRWho will winRetweet  #CSKLike         #RR'),
   (9, 'Dhoni appealed for lbw I guess #RRvCSK #IPL2020'),
   (9, 'Rajasthan finish with 2

In [ ]:
#solution of prob 4
tag_month_twit_monthwise_list = tag_month_twit_monthwise.collect()
print(tag_month_twit_monthwise_list)

[('#rrvcsk', [(9, '#IPL2020 CSK win toss and bowl #IPL #RRvCSK'), (9, 'Predict the expected run by rajasthanroyals #IPL2020 #RRvCSK #PredictionContest #HallaBol #Yellove'), (9, 'Wholl get his century first #RRvCSK #IPL2020 #Yellove #CSKvRR'), (9, 'I want Samson playing for India in all formats ASAP#IPL2020 #RRvCSK rajasthanroyals'), (9, 'Today match RR Vs CSK cricbuzz #RRvCSK #ipl2020'), (9, 'MSD not Archer #RRvCSK #IPL2020'), (9, 'Samson you beauty  #IPL2020 #RRvCSK #Dream11'), (9, 'Shitty umpiring #IPL2020 #IPLinUAE #RRvCSK ##rr #Csk'), (9, 'Jofstradamus has a tweet for everything  #IPL2020 #RRvCSK httpstcoDaVoi3erzC'), (9, 'Pic says everything  #CSKvsRR #RRvCSK #CSK #Cricket #IPL2020 #HallaBol #RCB #RCBvSRH #PlayBold #UmeshYadav httpstcoLMq2MqLlIy'), (9, 'Poll time #IPL2020 #RRvCSK #CSKvsRRWho will winRetweet  #CSKLike         #RR'), (9, 'Dhoni appealed for lbw I guess #RRvCSK #IPL2020'), (9, 'Rajasthan finish with 216 runs #RRvCSK #IPL2020'), (9, 'StarSportsIndia Its obviously grea

# Our Own Insights

## Insight 1:
### checking monthwise which location has maximum number of tweets

In [ ]:
#Building function to get tokenized tuple in form of (month,(user_location,1))
def extract_location_info(twit_row):
    if twit_row[5] in ('', ' ', '  '):
        return_tuple = ((parse_date(twit_row), '<Location Null>'), 1)
    else:
        return_tuple = (parse_date(twit_row), (twit_row[5].lower(), 1))
    return return_tuple


In [ ]:
#applying tokenization funtion
location_token = twit_tokenize_1.map(extract_location_info)

In [ ]:
location_token.take(2)

Out[39]: [(8, ('melbourne  new delhi', 1)), ((9, '<Location Null>'), 1)]

In [ ]:
#building funtion to get month-wise,location, number of tweets for a given month
def month_wise_location(month):
    # Filter dataset to get specific month rows
    month_wise = location_token.filter(lambda x: x[0] == month)
    
    # Get tuples in index 1 of extracted tokens
    month_wise_location_tag = month_wise.map(lambda rec: rec[1])
    
    # Get total number of tweets for a given location in a given month
    month_wise_tags_list = (
        month_wise_location_tag.groupByKey()
        .mapValues(list)
        .mapValues(sum)
    )
    
    # Sort in descending order to get location and their tweet counts
    month_wise_tags_list = month_wise_tags_list.sortBy(
        lambda x: x[1], ascending=False
    )
    
    # Get top 5 locations (hardcoded for top 5)
    top_rank = month_wise_tags_list.take(5)
    
    # Format the result as a list of [month, location, count] tuples
    result = list(map(lambda x: [month, x[0], x[1]], top_rank))
    
    return result


In [ ]:
ans5 = []
# Loop will run on available months
for i in sorted(twit_tokens_extracted.groupByKey().mapValues(list).keys().collect()):
    result_prob_1 = month_wise_location(i)
    ans5 += result_prob_1

# Create a PrettyTable instance
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Month", "User Location", "Number of Tweets"]

# Add rows to the table
for month, user_location, count in ans5:
    table.add_row([month, user_location, count])

# Print the table
print(table)


+-------+-----------------+------------------+
| Month |  User Location  | Number of Tweets |
+-------+-----------------+------------------+
|   8   |      india      |       211        |
|   8   |   mumbai india  |        82        |
|   8   | new delhi india |        82        |
|   8   |    new delhi    |        34        |
|   8   |  chennai india  |        31        |
|   9   |      india      |       2495       |
|   9   |   mumbai india  |       940        |
|   9   | new delhi india |       656        |
|   9   | hyderabad india |       495        |
|   9   | bengaluru india |       318        |
|   10  |      india      |       2168       |
|   10  |   mumbai india  |       835        |
|   10  | new delhi india |       560        |
|   10  | hyderabad india |       341        |
|   10  | bengaluru india |       271        |
|   11  |      india      |       739        |
|   11  |   mumbai india  |       329        |
|   11  | new delhi india |       201        |
|   11  | hyd


## Insight 2
### Which team has maximum team from which location

In [ ]:
#Building function to get tokens in form of (team,location,1)
def extract_team_refs_location(twit_row):
    team_location_includes = []
    tweet = twit_row[2]
    
    for team in ipl_teams_2020:
        if team in tweet:
            if twit_row[5] in ('', ' ', '  '):
                team_location_includes.append((team, ('<Location Null>', 1)))
            else:
                team_location_includes.append((team, (twit_row[5], 1)))
    
    return team_location_includes


In [ ]:
#applying tokenization function
data_team = twit_tokenize_1.flatMap(extract_team_refs_location)

In [ ]:
data_team.take(10)

Out[45]: [('CSK', ('<Location Null>', 1)),
 ('RR', ('<Location Null>', 1)),
 ('RCB', ('<Location Null>', 1)),
 ('KXIP', ('<Location Null>', 1)),
 ('DC', ('India', 1)),
 ('KXIP', ('India', 1)),
 ('KKR', ('<Location Null>', 1)),
 ('DC', ('<Location Null>', 1)),
 ('RCB', ('Bengaluru India', 1)),
 ('KXIP', ('Bengaluru India', 1))]

In [ ]:
#Building function to get 
def team_wise_location_max(team_ticker):
    # Filter dataset to get team ticker specific rows
    teamwise_data = data_team.filter(lambda x: x[0] == team_ticker)
    
    # Get tuples in index 1 of extracted tokens
    teamwise_location_tuple = teamwise_data.map(lambda rec: rec[1])
    
    # Get total number of tweets for a given team in a given location
    team_wise_location_rdd = (
        teamwise_location_tuple.groupByKey()
        .mapValues(list)
        .mapValues(sum)
    )
    
    # Sort in descending order to get location and their tweet counts
    team_wise_location_rdd = team_wise_location_rdd.sortBy(
        lambda x: x[1], ascending=False
    )
    
    # Get top 4 locations
    team_wise_location_result_tuple = team_wise_location_rdd.take(4)
    
    return team_wise_location_result_tuple


In [ ]:
#printing results
ans3 = []
for team in ipl_teams_2020:
    temp = team_wise_location_max(team)
    ans3.append((team, (temp[0], temp[1], temp[2], temp[3])))

for team, tweets in ans3:
    print(f"Team: {team}")
    for location, count in tweets:
        print(f"Has {count} tweets from location: {location}")
    print()


Team: MI
Has 1969 tweets from location: <Location Null>
Has 796 tweets from location: India
Has 389 tweets from location: Mumbai India
Has 182 tweets from location: New Delhi India

Team: CSK
Has 2147 tweets from location: <Location Null>
Has 792 tweets from location: India
Has 297 tweets from location: Mumbai India
Has 222 tweets from location: New Delhi India

Team: RCB
Has 2224 tweets from location: <Location Null>
Has 890 tweets from location: India
Has 375 tweets from location: Mumbai India
Has 250 tweets from location: New Delhi India

Team: KKR
Has 1188 tweets from location: <Location Null>
Has 565 tweets from location: India
Has 206 tweets from location: Mumbai India
Has 141 tweets from location: New Delhi India

Team: SRH
Has 1546 tweets from location: <Location Null>
Has 671 tweets from location: India
Has 255 tweets from location: Mumbai India
Has 185 tweets from location: Hyderabad India

Team: DC
Has 1609 tweets from location: <Location Null>
Has 773 tweets from location: 

## Insight  3
### Which proportion of source in the tweets

In [ ]:
#Building function to get tokens in form of (source,1)
def tweet_source(twit_row):
    return (twit_row[6],1)

In [ ]:
twit_by_source = twit_tokenize_1.map(tweet_source)

In [ ]:
#count tweet by source by adding key-value pairs
twit_by_source_count=twit_by_source.reduceByKey(lambda a,b:a+b)

In [ ]:
#getting a list from rdd and sorting it based on number of tweets
twit_by_source_count_list = twit_by_source_count.collect()
twit_by_source_count_list_sorted = sorted(
    twit_by_source_count_list, key=lambda x: x[1], reverse=True
)


In [ ]:
#printing results
for source, count in twit_by_source_count_list_sorted:
    print(f'We got {count} tweets from "{source}"')


We got 29416 tweets from "Twitter for Android"
We got 10916 tweets from "Twitter Web App"
We got 5690 tweets from "Twitter for iPhone"
We got 2570 tweets from "TweetDeck"
We got 184 tweets from "IPLT20 Live"
We got 182 tweets from "Hootsuite Inc."
We got 142 tweets from "Twitter for iPad"
We got 132 tweets from "Buffer"
We got 124 tweets from "Twitter Media Studio"
We got 70 tweets from "IFTTT"
We got 58 tweets from "The Cricket Times"
We got 48 tweets from "Instagram"
We got 47 tweets from "Twitter for Advertisers"
We got 45 tweets from "Republicworld"
We got 36 tweets from "Twitter for Advertisers (legacy)"
We got 27 tweets from "WordPress.com"
We got 21 tweets from "TweetCaster for Android"
We got 21 tweets from "Mobile Web (M2)"
We got 18 tweets from "Wildmoka"
We got 15 tweets from "My Khel"
We got 14 tweets from "Hocalwire Social Share"
We got 12 tweets from "Plume¬†for¬†Android"
We got 12 tweets from "Publer "
We got 11 tweets from "Twitter Media Studio - LiveCut"
We got 10 twee